In [1]:
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
rcParams['figure.figsize'] = (7,7)

import random
import tensorflow as tf

seed = 42


random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)


# Import deepscore model # Add deepscore folder to path
from deepscore import deepscore 
import pickle

2025-08-14 17:40:15.646607: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-14 17:40:15.803079: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-14 17:40:15.804029: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-14 17:40:16.533597: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
cell_type = 'cell_type'

ref_py = sc.read('/home/macera/Documentos/CZI/MANUSCRIPT_PREP/REVIEWS/external_data/heart/reference.h5ad')


In [3]:
ref_py.layers['counts'] = ref_py.X.copy()

ref_py.var['ENSG'] = ref_py.var.index.copy()
# ref_py.var.index = ref_py.var['feature_name'].copy()

In [4]:
ref_py.X.data

array([1., 1., 1., ..., 1., 2., 1.], dtype=float32)

In [5]:

overlapping = False
compute = False


ref_py.X = ref_py.X.copy()
sc.pp.normalize_total(ref_py, target_sum=1e4)
sc.pp.log1p(ref_py)


markers_filename= f'heart_cell_type'

# Identify HCA ATLAS differentially expressed genes between cell types 

if compute == True:
    sc.tl.rank_genes_groups(ref_py, cell_type, method='wilcoxon', use_raw=False)
    ranked_genes_populations = ref_py.uns['rank_genes_groups'].copy()
    with open(f'markers_ds/{markers_filename}.pickle', 'wb') as handle:
        pickle.dump(ranked_genes_populations, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(f'markers_ds/{markers_filename}.pickle', 'rb') as handle:
        ranked_genes_populations = pickle.load(handle) 

In [6]:
markers_filename= f'heart_cell_type'

# Identify HCA ATLAS differentially expressed genes between cell types 

if compute == True:
    sc.tl.rank_genes_groups(ref_py, cell_type, method='wilcoxon', use_raw=False)
    ranked_genes_populations = ref_py.uns['rank_genes_groups'].copy()
    with open(f'markers_ds/{markers_filename}.pickle', 'wb') as handle:
        pickle.dump(ranked_genes_populations, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(f'markers_ds/{markers_filename}.pickle', 'rb') as handle:
        ranked_genes_populations = pickle.load(handle) 

In [7]:
adata =  sc.read('/home/macera/Documentos/CZI/MANUSCRIPT_PREP/REVIEWS/external_data/heart/query.h5ad')
adata

AnnData object with n_obs × n_vars = 229621 × 32732
    obs: 'sangerID', 'combinedID', 'donor', 'donor_type', 'region', 'region_finest', 'age', 'gender', 'facility', 'cell_or_nuclei', 'modality', 'kit_10x', 'flushed', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'scrublet_score', 'scrublet_leiden', 'cluster_scrublet_score', 'doublet_pval', 'doublet_bh_pval', 'batch_key', 'leiden_scVI', 'cell_type', 'cell_state_HCAv1', 'cell_state_scNym', 'cell_state_scNym_confidence', 'cell_state', 'latent_RT_efficiency', 'latent_cell_probability', 'latent_scale', 'n_counts', '_scvi_batch', '_scvi_labels', 'clus20', 'doublet_cls', 'original_or_new', 'batch', 'scANVI_predictions', 'leiden_scArches', 'Tech'
    var: 'gene_name-new', 'gene_name_scRNA-0-original', 'gene_name_snRNA-1-original', 'gene_name_multiome-2-original'
    uns: 'age_colors', 'cell_or_nuclei_colors', 'cell_state_colors', 'cell_type_colors', 'donor_colors', 

In [ ]:
## SET PARAMETERS
n_markers = 500 # Max number of markers to use per cell-type
overlapping = False # Parameter to control overlapping marker genes between cell types on the prediction.


# ref_py_save = ref_py.copy()

# for mod in ['scRNA','snRNA','scRNA5p']:
#     if os.path.exists(f'csv/Deepscore_HCA_l1_{mod}_CLEAN.csv'):
#         print(f'{mod} already exists!')

mod='HEART'

# adata.X = adata.layers['counts'].copy()


with open(f'markers_ds/{markers_filename}.pickle', 'rb') as handle:
    ranked_genes_populations = pickle.load(handle) 

if overlapping:
    selected_markers =[]
    for cell_type_ in ref_py.obs[cell_type].unique():
        cell_type_markers = []
        for marker in ranked_genes_populations['names'][cell_type_][:n_markers]:
            if marker in adata.var.index: 
                selected_markers.append(marker)
    selected_markers = set(selected_markers)

else:
    # Step 2: Store markers for each subset
    subset_markers_dict ={}
    for subset in ref_py.obs[cell_type].unique():
        subset_markers = ranked_genes_populations['names'][subset]
        subset_markers = [gene for gene in subset_markers if gene in adata.var.index]
        subset_markers_dict[subset] = set(subset_markers[:n_markers+100])

    # Step 3: Identify overlapping markers
    overlapping_markers = set()
    for subset, markers in subset_markers_dict.items():
        for other_subset, other_markers in subset_markers_dict.items():
            if subset != other_subset:
                overlapping_markers.update(markers.intersection(other_markers))

    # Step 4: Select markers for each subset, excluding overlapping markers
    marker_dict = {}
    for subset, markers in subset_markers_dict.items():
        unique_markers = [marker for marker in markers if marker not in overlapping_markers]
        marker_dict[subset] = unique_markers[:n_markers]  # Select up to TOP n_markers
    selected_markers = [marker for subset in marker_dict for marker in marker_dict[subset]]



    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)


    # Subset the data to the selected markers

    ref_py = ref_py[:, list(selected_markers)].copy()
    adata = adata[:, list(selected_markers)].copy()

    len(selected_markers)

    sc.pp.scale(ref_py)
    sc.pp.scale(adata)

    ref_py.obs[cell_type] = ref_py.obs[cell_type].tolist()
    len(ref_py.obs[cell_type].unique())


    def scheduler(epoch, lr):
        if epoch < 10:
            return lr
        else:
            return lr * tf.math.exp(-0.1)


    n_feat = ref_py.shape[1]
    n_labs = len(ref_py.obs[cell_type].unique())

    ds = deepscore.DeepScore(hidden_nodes=[1024, 256],
                   n_features=n_feat, 
                   n_labels=n_labs,
                   epochs=30,
                   batch_size=128, 
                   activation="relu", 
                   dropout=True, 
                   dropout_rate=0.1,
                   batchnorm=True, 
                   lr=0.001,
                   weight_reg=True)




    import os
    os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

    ds.set_reference(ref_py, label_by=cell_type, test_prop=0.1)

    ds.train(earlystopping=True, patience=10, lr_scheduler=scheduler,)
    # ds.model.save(f'models/deepscore') # In case you want to save the DS model

    prob_df, adata = ds.annotate(adata, pred_key='Deepscore_HCA',Unclassified = False,return_pred_matrix=True)

    # SAVE the RESULTS on csv
    adata.obs[['Deepscore_HCA','Deepscore_HCA_score']].to_csv(f'csv/Deepscore_HCA_l1_{mod}.csv')

    prob_df.to_csv(f'csv/prob_matrix/Deepscore_HCA_l1_{mod}_CLEAN.csv')


Model: "deepscore"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (Batch  (None, 1850)              7400      
 Normalization)                                                  
                                                                 
 dense1024 (Dense)           (None, 1024)              1895424   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 batch_normalization_1 (Bat  (None, 1024)              4096      
 chNormalization)                                                
                                                                 
 dense256 (Dense)            (None, 256)               262400    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0 

2025-08-14 17:43:00.721280: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2488464600 exceeds 10% of free system memory.
2025-08-14 17:43:01.785926: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2239617400 exceeds 10% of free system memory.


Epoch 1/30
2365/2365 [==============================] - 59s 25ms/step - loss: 0.0065 - categorical_accuracy: 0.9979 - val_loss: 0.0029 - val_categorical_accuracy: 0.9991 - lr: 0.0010
Epoch 2/30
2365/2365 [==============================] - 66s 28ms/step - loss: 0.0027 - categorical_accuracy: 0.9991 - val_loss: 0.0032 - val_categorical_accuracy: 0.9991 - lr: 0.0010
Epoch 3/30
2365/2365 [==============================] - 66s 28ms/step - loss: 0.0020 - categorical_accuracy: 0.9993 - val_loss: 0.0034 - val_categorical_accuracy: 0.9990 - lr: 0.0010
Epoch 4/30
2365/2365 [==============================] - 70s 30ms/step - loss: 0.0022 - categorical_accuracy: 0.9992 - val_loss: 0.0021 - val_categorical_accuracy: 0.9993 - lr: 0.0010
Epoch 5/30
2365/2365 [==============================] - 70s 30ms/step - loss: 0.0021 - categorical_accuracy: 0.9993 - val_loss: 0.0028 - val_categorical_accuracy: 0.9993 - lr: 0.0010
Epoch 6/30
2365/2365 [==============================] - 71s 30ms/step - loss: 0.0019 